In [1]:
!pip install transformers
!pip install ipywidgets
!pip install IProgress
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [69]:
import transformers
from transformers import BertTokenizer, BertConfig
from transformers import BertModel, BertForSequenceClassification
from transformers import BatchEncoding



import torch
import torch.nn as nn
from torch.utils.data import dataloader

import datasets

## tokenizer

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

## data

In [4]:
# DATA_FOLDER = '/notebooks/Data/bert_sequence_classification'
DATA_FILE = '/notebooks/KURI-BERT/data/pe_dataset_for_bert_minus.pt'
# RESULTS_FOLDER = '/notebooks/Results/bert_sequence_classification'

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
device

device(type='cuda')

## Load data

In [7]:
dataset = torch.load(DATA_FILE)

In [8]:
dataset

Dataset({
    features: ['paragraph', 'paragraph_ACs', 'paragraph_AMs', 'paragraph_labels', 'paragraph_ac_spans', 'paragraph_am_spans', 'split', 'essay_nr'],
    num_rows: 1719
})

In [77]:
dataset['paragraph_am_spans']

[[[77, 81]],
 [[0, 2], [-1, -1], [-1, -1], [-1, -1]],
 [[0, 3], [21, 21], [34, 34], [-1, -1], [-1, -1]],
 [[0, 0]],
 [[54, 54]],
 [[0, 0], [-1, -1], [-1, -1], [-1, -1], [101, 101]],
 [[0, 0], [-1, -1], [31, 32], [-1, -1], [-1, -1], [101, 101]],
 [[-1, -1], [23, 28]],
 [[-1, -1]],
 [[0, 0], [-1, -1], [-1, -1], [-1, -1]],
 [[0, 0], [19, 20], [-1, -1], [79, 79]],
 [[3, 8]],
 [[-1, -1], [-1, -1]],
 [[0, 0], [-1, -1], [-1, -1], [-1, -1]],
 [[0, 0], [-1, -1], [-1, -1], [100, 100]],
 [[3, 6]],
 [[45, 47]],
 [[-1, -1], [-1, -1]],
 [[-1, -1], [-1, -1], [-1, -1], [58, 58]],
 [[-1, -1], [-1, -1], [30, 31]],
 [[0, 1], [17, 17]],
 [[19, 21], [-1, -1]],
 [[0, 2], [-1, -1], [42, 43], [55, 56], [-1, -1]],
 [[-1, -1], [24, 25], [-1, -1]],
 [[-1, -1],
  [-1, -1],
  [29, 29],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 [[0, 1]],
 [[46, 46]],
 [[0, 2], [-1, -1], [-1, -1], [103, 103]],
 [[-1, -1], [-1, -1], [-1, -1], [-1, -1]],
 [[2, 2], [-1, -1], [-1, -1]],
 [[-1, -1]],
 [[-1, -1], [-1, 

In [23]:
dataset['paragraph_ac_spans']

[[[87, 96]],
 [[4, 23], [25, 51], [53, 94], [96, 116]],
 [[5, 19], [27, 32], [36, 56], [70, 131], [148, 158]],
 [[24, 37]],
 [[59, 75]],
 [[2, 21], [23, 55], [57, 75], [77, 99], [107, 115]],
 [[2, 10], [12, 29], [34, 48], [50, 78], [80, 99], [107, 117]],
 [[4, 21], [30, 50]],
 [[48, 66]],
 [[2, 21], [23, 55], [57, 79], [82, 90]],
 [[2, 17], [22, 57], [59, 77], [81, 92]],
 [[10, 26]],
 [[21, 29], [35, 50]],
 [[10, 25], [27, 76], [78, 97], [103, 112]],
 [[8, 20], [22, 69], [75, 98], [106, 111]],
 [[7, 20]],
 [[49, 59]],
 [[2, 25], [27, 63]],
 [[2, 23], [25, 42], [44, 56], [60, 69]],
 [[2, 13], [15, 28], [32, 50]],
 [[3, 15], [19, 38]],
 [[22, 26], [28, 42]],
 [[4, 20], [22, 40], [45, 53], [57, 69], [71, 84]],
 [[5, 22], [27, 35], [37, 56]],
 [[0, 8],
  [10, 27],
  [31, 38],
  [40, 57],
  [59, 66],
  [68, 82],
  [84, 97],
  [100, 110]],
 [[3, 20]],
 [[63, 73]],
 [[4, 18], [20, 61], [63, 101], [105, 119]],
 [[5, 26], [34, 49], [85, 97], [103, 126]],
 [[6, 29], [31, 47], [53, 67]],
 [[78, 8

### dataloader

In [70]:
loader = dataloader.DataLoader(dataset, 32, shuffle=False)

In [71]:
batch = next(iter(loader))

RuntimeError: each element in list of batch should be of equal size

In [64]:
# ok this is because the paras, spans etc are not of the same lenght

In [65]:
# here we could do with a collate functions

In [ ]:
def custom_collate(data):
    inputs = [torch.tensor(['tokenized_input']) for d in data]
    labels = [d['label'] for d in data]
    inputs = pad_sequence(inputs, batch_first=True)
    labels = torch.tensor(labels)
    return {
        'tokenized_input': inputs, 
        'label': labels
    }

In [ ]:
# ac_spans == am_spans but they are different for every paragraph we need to make them uniform.

In [22]:
# we need some padding

In [24]:
# try a function

In [36]:
def am_span_padding(dataset):
    
    max_length = max([len(l) for l in dataset['paragraph_am_spans']])
    
    padded_spans = []

    for idx, span in enumerate(dataset['paragraph_am_spans']):

        padded_span = dataset['paragraph_am_spans'][idx] + (max_length - len(span)) * [[-1,-1]]
        padded_spans.append(padded_span)
        
    return padded_spans

In [31]:
am_span_padding(dataset)

[[[77, 81],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 [[0, 2],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 [[0, 3],
  [21, 21],
  [34, 34],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 [[0, 0],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 [[54, 54],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 [[0, 0],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [101, 101],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 [[0, 0],
  [-1, -1],
  [31, 32],
  [-1, -1],
  [-1, -1],
  [101, 101],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-

In [40]:
def ac_span_padding(dataset):
    
    max_length = max([len(l) for l in dataset['paragraph_ac_spans']])
    
    padded_spans = []

    for idx, span in enumerate(dataset['paragraph_ac_spans']):

        padded_span = dataset['paragraph_ac_spans'][idx] + (max_length - len(span)) * [[-1,-1]]
        padded_spans.append(padded_span)
        
    return padded_spans

In [33]:
ac_span_padding(dataset)

[[[87, 96],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 [[4, 23],
  [25, 51],
  [53, 94],
  [96, 116],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 [[5, 19],
  [27, 32],
  [36, 56],
  [70, 131],
  [148, 158],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 [[24, 37],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 [[59, 75],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 [[2, 21],
  [23, 55],
  [57, 75],
  [77, 99],
  [107, 115],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1]],
 [[2, 10],
  [12, 29],
  [34, 48],
  [50, 78],
  [80, 99],
  [107, 117],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1, -1],
  [-1,

In [49]:
def labels_padding(dataset):
    
    max_length = max([len(l) for l in dataset['paragraph_labels']])
    
    padded_labels_l = []

    for idx, labels in enumerate(dataset['paragraph_labels']):

        padded_labels = dataset['paragraph_labels'][idx] + (max_length - len(labels)) * [-1]
        padded_labels_l.append(padded_labels)
        
    return padded_labels_l

In [50]:
label_padding(dataset)

[[0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [1, 2, 2, 2, -1, -1, -1, -1, -1, -1, -1, -1],
 [2, 1, 2, 2, 1, -1, -1, -1, -1, -1, -1, -1],
 [0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [2, 2, 2, 2, 1, -1, -1, -1, -1, -1, -1, -1],
 [2, 2, 2, 2, 2, 1, -1, -1, -1, -1, -1, -1],
 [1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [2, 2, 2, 1, -1, -1, -1, -1, -1, -1, -1, -1],
 [2, 2, 2, 1, -1, -1, -1, -1, -1, -1, -1, -1],
 [0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [2, 2, 2, 1, -1, -1, -1, -1, -1, -1, -1, -1],
 [2, 2, 2, 1, -1, -1, -1, -1, -1, -1, -1, -1],
 [0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [1, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [1, 2, 2, 2, -1, -1, -1, -1, -1, -1, -1, -1],
 [2, 2, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [1, 0, -1, -1, -1, -1, -1, -1, -1, 

In [ ]:
# yeah. the HF thing where you put -100 as the dummy

In [ ]:
# I think correct now.

In [17]:
### tokenize

In [51]:
def tokenize(dataset):
    
    tokenized_text = tokenizer(dataset['paragraph'], padding=True, return_tensors="pt")
    tokenized_text['am_spans'] = torch.tensor(am_span_padding(dataset))
    tokenized_text['ac_spans'] = torch.tensor(ac_span_padding(dataset))
    tokenized_text['labels'] = torch.tensor(label_padding(dataset))
    return tokenized_text

In [52]:
tokenized_input = tokenize(dataset)

In [53]:
tokenized_input

{'input_ids': tensor([[ 101, 2009, 2003,  ...,    0,    0,    0],
        [ 101, 2034, 1997,  ...,    0,    0,    0],
        [ 101, 2006, 1996,  ...,    0,    0,    0],
        ...,
        [ 101, 2006, 1996,  ...,    0,    0,    0],
        [ 101, 2012, 1996,  ...,    0,    0,    0],
        [ 101, 1999, 7091,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'am_spans': tensor([[[77, 81],
         [-1, -1],
         [-1, -1],
         ...,
         [-1, -1],
         [-1, -1],
         [-1, -1]],

        [[ 0,  2],
         [-1, -1],
     

In [55]:
tokenized_input['input_ids'].shape

torch.Size([1719, 200])

In [56]:
tokenized_input['am_spans'].shape

torch.Size([1719, 12, 2])

In [57]:
tokenized_input['ac_spans'].shape

torch.Size([1719, 12, 2])

In [58]:
tokenized_input['labels'].shape

torch.Size([1719, 12])

In [ ]:
# do we really need spans jointly? can we change the custom class to have am spans and ac spans separately?

### dataloader

In [12]:
### maybe I need a collate fn

In [ ]:
### collate functions

In [67]:
nlp_data = [
    {'tokenized_input': [1, 4, 5, 9, 3, 2],
     'label':0},
    {'tokenized_input': [1, 7, 3, 14, 48, 7, 23, 154, 2],
     'label':0},
    {'tokenized_input': [1, 30, 67, 117, 21, 15, 2],
     'label':1},
    {'tokenized_input': [1, 17, 2],
     'label':0},
]

In [73]:
loader = dataloader.DataLoader(nlp_data, batch_size=2, shuffle=False)
batch = next(iter(loader))

RuntimeError: each element in list of batch should be of equal size

In [ ]:
def custom_collate(data):
    inputs = [torch.tensor(d['tokenized_input']) for d in data]
    labels = [d['label'] for d in data]
    inputs = pad_sequence(inputs, batch_first=True)
    labels = torch.tensor(labels)
    return {
        'tokenized_input': inputs, 
        'label': labels
    }

In [14]:
def collate_fn(self, batch):
    
    batch_d = {}

    # texts
    texts_l = [sample["text"] for sample in batch]

    with torch.no_grad():

        text_tkn = self.tokenizer(texts_l, padding=True)
        text_tkn = torch.tensor(text_tkn["input_ids"]).to(self.device)

        batch_d["text"] = text_tkn

    # labels
    batch_d["label"] = torch.tensor([sample["label"] for sample in batch]).to(self.device)

    return batch_d